# [19、Transformer模型Encoder原理精讲及其PyTorch逐行实现](https://www.bilibili.com/video/BV1cP4y1V7GF)

## 难点

- word embedding
- position embedding
- encode self-attention mask
- intra-attention mask
- decoder self-attention mask
- multi-head self-attention

项目推荐：
- vit
- swing transformer
- nlp：预训练
- huggingface

![image-20250624205950984](http://assets.hypervoid.top/img/2025/06/24/image-20250624205950984-f71b.png)

In [2]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F


## Word Embedding

以序列建模为例子考虑 source sentence 和 target sentence 

In [3]:
# 构建序列，序列字符以在词表中索引的形式
from torch import int32

num_words = 8 # 单词表大小
model_dim = 8 # 模型特征大小
batch_size = 2
# 代表句子长度
src_len, tgt_len = [2,4], [4,3]
# src_seq = [torch.randint(1, num_words, (L, )) for L in src_len]
src_seq = torch.zeros((batch_size, max(src_len)), dtype=int32)
tgt_seq = torch.zeros((batch_size, max(tgt_len)), dtype=int32)
for i in range(batch_size):
    _len = int(src_len[i])
    src_seq[i, :_len] = torch.randint(1, num_words, (_len,))
    _len = int(tgt_len[i])
    tgt_seq[i, :_len] = torch.randint(1, num_words, (_len,))

src_emb_table = nn.Embedding(num_words+1, model_dim)
tgt_emb_table = nn.Embedding(num_words+1, model_dim)

# src_len, tgt_len
print("src_seq = ", src_seq)
print("tgt_seq = ", tgt_seq)
print("emb_table=", src_emb_table.weight)

# 构造 word embedding
src_emb = src_emb_table(src_seq) # 就是算出每个label对应的向量
tgt_emb = tgt_emb_table(src_seq)
src_emb, src_emb.shape

src_seq =  tensor([[4, 2, 0, 0],
        [5, 5, 1, 4]], dtype=torch.int32)
tgt_seq =  tensor([[1, 7, 3, 7],
        [5, 2, 6, 0]], dtype=torch.int32)
emb_table= Parameter containing:
tensor([[-0.1615,  1.0848, -0.7028,  1.0415, -0.4725, -1.1625,  1.0262,  1.0964],
        [-1.0604, -0.1099, -0.7894, -0.8810, -0.2279,  0.7442,  1.2569,  1.0380],
        [-0.4024,  0.8135, -0.9275, -1.3547, -1.0342,  0.7953,  0.0574, -0.0495],
        [ 0.2147,  0.1891,  0.4006,  0.7699,  1.3729,  0.3016, -1.2966,  0.4621],
        [-1.3259,  1.9471, -1.2459, -0.4742, -0.2590, -1.0082, -0.3390, -0.4506],
        [-2.2342,  0.8628,  0.0396, -0.4539,  0.0152, -0.9146, -0.7273,  0.8294],
        [ 0.5619, -0.8959, -0.5630, -1.0634,  0.3173,  0.7419,  2.1825,  0.9669],
        [ 0.6489,  0.3962,  0.9375,  0.2318,  0.7550, -0.4951,  0.0103, -0.0439],
        [ 0.4689, -0.6901, -1.6390, -1.0486,  0.0407,  1.2266,  0.1206, -1.2306]],
       requires_grad=True)


(tensor([[[-1.3259,  1.9471, -1.2459, -0.4742, -0.2590, -1.0082, -0.3390,
           -0.4506],
          [-0.4024,  0.8135, -0.9275, -1.3547, -1.0342,  0.7953,  0.0574,
           -0.0495],
          [-0.1615,  1.0848, -0.7028,  1.0415, -0.4725, -1.1625,  1.0262,
            1.0964],
          [-0.1615,  1.0848, -0.7028,  1.0415, -0.4725, -1.1625,  1.0262,
            1.0964]],
 
         [[-2.2342,  0.8628,  0.0396, -0.4539,  0.0152, -0.9146, -0.7273,
            0.8294],
          [-2.2342,  0.8628,  0.0396, -0.4539,  0.0152, -0.9146, -0.7273,
            0.8294],
          [-1.0604, -0.1099, -0.7894, -0.8810, -0.2279,  0.7442,  1.2569,
            1.0380],
          [-1.3259,  1.9471, -1.2459, -0.4742, -0.2590, -1.0082, -0.3390,
           -0.4506]]], grad_fn=<EmbeddingBackward0>),
 torch.Size([2, 4, 8]))

## Position Embedding

Transformer没有位置假设和全局假设，因此需要位置信息。

$$
PE(pos, 2i) = \sin(\frac{pos}{10000^{2i/d_{\text{model}}}})\\
PE(pos, 2i+1) = \cos(\frac{pos}{10000^{2i/d_{\text{model}}}})
$$

使用Sin和Cos组合的位置编码，泛化能力比较强，具有对称性，具有唯一性。
- 唯一性：每个位置的编码独一无二
- 编码值在不同维度上，有不同的变化频率
- 相对位置的线性表示：可以将 PE(pos+k, 2i) 和 PE(pos+k, 2i+1) 表示成 PE(pos, 2i) 和 PE(pos, 2i+1) 的线性组合。这等价于将位置 pos 的编码向量乘以一个不依赖于 pos 而只依赖于 k 的旋转矩阵.
    - 因此模型不需要去学习每个绝对位置的含义，只需要学习这个“旋转”操作的含义。例如，模型可以学会一个通用的变换，来关注“前面第k个词”或“后面第k个词”，无论当前的绝对位置 pos 是多少。这使得模型能够轻松地推断出相对位置关系，这对于理解语言至关重要。


$$
\sin(A+B) = \sin(A)\cos(B) + \cos(A)\sin(B) \\
\cos(A+B) = \cos(A)\cos(B) - \sin(A)\sin(B)
$$

In [4]:
# 构建pos embedding
max_pos = 8
pos = torch.arange(max_pos).reshape((-1, 1)) # 一列
# 总model_dim
idx = torch.arange(0, model_dim).reshape((1, -1)) # 一行
inner = pos / torch.pow(10000, idx / model_dim)
pos_emb = torch.empty_like(inner)
inner = inner[:, 0::2] # 只取i的偶数部分
pos_emb[:, 0::2] = torch.sin(inner)
pos_emb[:, 1::2] = torch.cos(inner)
pelayer = nn.Embedding(pos_emb.shape[0], pos_emb.shape[1])
pelayer.weight = nn.Parameter(pos_emb, requires_grad=False)
# 现在构造上面句子的 pos embedding

src_pos = torch.arange(max(src_len)).repeat(batch_size, 1)
tgt_pos = torch.arange(max(tgt_len)).repeat(batch_size, 1)
src_pe = pelayer(src_pos)
tgt_pe = pelayer(tgt_pos)
src_pe.shape, tgt_pe.shape

(torch.Size([2, 4, 8]), torch.Size([2, 4, 8]))

## encoder self-attention mask

用于加速，一次训练多对样本序列

### 为什么要scale？

In [5]:
#  softmax 的马太效应

arr = torch.arange(0, 3, dtype=torch.float32)
print(F.softmax(arr, dim=0))
print(F.softmax(arr * 3, dim=0))
print(F.softmax(arr * 5, dim=0))
# 在不同的 scale 下，softmax 出来的分布越来越尖锐。模型就不会更关心小一些的值了

tensor([0.0900, 0.2447, 0.6652])
tensor([0.0024, 0.0473, 0.9503])
tensor([4.5094e-05, 6.6925e-03, 9.9326e-01])


雅可比矩阵角度，scale越大，雅可比矩阵反而越小，导致训练的时候梯度变小，模型难以训练。

In [6]:
# 雅可比角度
from torch import Tensor
from torch.autograd.functional import jacobian


def softmax_f(arr):
    return F.softmax(arr, dim=0)


res1: Tensor = jacobian(softmax_f, arr)
res2: Tensor = jacobian(softmax_f, arr * 5)
res3: Tensor = jacobian(softmax_f, arr * 10)

print(res1)
print(res2)
print(res3)
print(res1.abs().sum(), res2.abs().sum(), res3.abs().sum())


tensor([[ 0.0819, -0.0220, -0.0599],
        [-0.0220,  0.1848, -0.1628],
        [-0.0599, -0.1628,  0.2227]])
tensor([[ 4.5092e-05, -3.0179e-07, -4.4790e-05],
        [-3.0179e-07,  6.6478e-03, -6.6475e-03],
        [-4.4790e-05, -6.6475e-03,  6.6923e-03]])
tensor([[ 2.0611e-09, -9.3568e-14, -2.0610e-09],
        [-9.3568e-14,  4.5396e-05, -4.5396e-05],
        [-2.0610e-09, -4.5396e-05,  4.5417e-05]])
tensor(0.9789) tensor(0.0268) tensor(0.0002)


### mask

In [7]:
# mask大小：[batch_size, max_src_len, max_src_len]，值 0/-inf
print(src_seq)
valid_encoder_pos = torch.where(src_seq != 0, 1, 0)
print(valid_encoder_pos)
valid_encoder_pos=valid_encoder_pos.unsqueeze_(2)
valid_encoder_mat = torch.bmm(valid_encoder_pos, valid_encoder_pos.transpose(1,2))
# 获得mask矩阵，代表能否建立关联性（和pad后的无关系）
mask = (1-valid_encoder_mat).to(torch.bool)
print(mask)
# mask用法：
score = torch.randn(mask.shape)
score.masked_fill_(mask, -np.inf)
prob = F.softmax(score)
print("prob: ", prob, prob.shape)

tensor([[4, 2, 0, 0],
        [5, 5, 1, 4]], dtype=torch.int32)
tensor([[1, 1, 0, 0],
        [1, 1, 1, 1]])
tensor([[[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True,  True,  True],
         [ True,  True,  True,  True]],

        [[False, False, False, False],
         [False, False, False, False],
         [False, False, False, False],
         [False, False, False, False]]])
prob:  tensor([[[0.1279, 0.1505, 0.0000, 0.0000],
         [0.1732, 0.2863, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.8721, 0.8495, 1.0000, 1.0000],
         [0.8268, 0.7137, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000, 1.0000]]]) torch.Size([2, 4, 4])


/tmp/ipykernel_91144/4008299438.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = F.softmax(score)


# [20、Transformer模型Decoder原理精讲及其PyTorch逐行实现](https://www.bilibili.com/video/BV1Qg411N74v?spm_id_from=333.788.player.switch&vd_source=cdd897fffb54b70b076681c3c4e4d45d)

## 位置编码的泛化能力

假如说我们训练时候使用512的长度，但是在实际使用的时候遇到了1000 的长度，这时候可以通过数学公式推导出对应的位置编码。
$$
M. \begin{bmatrix} \sin(\omega_k. t) \\ \cos(\omega_k. t) \end{bmatrix} = \begin{bmatrix} \sin(\omega_k. (t + \phi)) \\ \cos(\omega_k. (t + \phi)) \end{bmatrix}
$$

这里 $t+\phi$ 超过了限度，但是假如我们可以找到一个矩阵 $M$，使得：
$$
\begin{bmatrix} u_1 & v_1 \\ u_2 & v_2 \end{bmatrix} \begin{bmatrix} \sin(\omega_k . t) \\ \cos(\omega_k . t) \end{bmatrix} = \begin{bmatrix} \sin(\omega_k . (t + \phi)) \\ \cos(\omega_k . (t + \phi)) \end{bmatrix}
$$

然后借助和差化积公式解方程：

$$
\begin{bmatrix} u_1 & v_1 \\ u_2 & v_2 \end{bmatrix} \begin{bmatrix} \sin(\omega_k . t) \\ \cos(\omega_k . t) \end{bmatrix} = \begin{bmatrix} \sin(\omega_k . t) \cos(\omega_k . \phi) + \cos(\omega_k . t) \sin(\omega_k . \phi) \\ \cos(\omega_k . t) \cos(\omega_k . \phi) - \sin(\omega_k . t) \sin(\omega_k . \phi) \end{bmatrix}
$$

$$
u_1 \sin(\omega_k . t) + v_1 \cos(\omega_k . t) = \cos(\omega_k . \phi) \sin(\omega_k . t) + \sin(\omega_k . \phi) \cos(\omega_k . t) \quad (1) \\
u_2 \sin(\omega_k . t) + v_2 \cos(\omega_k . t) = - \sin(\omega_k . \phi) \sin(\omega_k . t) + \cos(\omega_k . \phi) \cos(\omega_k . t) \quad (2)
$$

解得：
$$
u_1 = \cos(\omega_k . \phi) \quad v_1 = \sin(\omega_k . \phi) \\
u_2 = - \sin(\omega_k . \phi) \quad v_2 = \cos(\omega_k . \phi)
$$


于是得到矩阵 $M$ 就是：

$$
M_{\phi,k} = \begin{bmatrix} \cos(\omega_k . \phi) & \sin(\omega_k . \phi) \\ - \sin(\omega_k . \phi) & \cos(\omega_k . \phi) \end{bmatrix}
$$

## Intra Attention Mask

计算 $Q \times K^T$, `shape = (batch_size, target_seq_len, src_seq_len)`

In [19]:
# 1 代表是不是真的单词，不是padding的
valid_encoder_pos = torch.where(src_seq != 0, 1, 0).unsqueeze(2)
# print(valid_encoder_pos.shape, valid_encoder_pos, sep='\n')

valid_decoder_pos = torch.where(tgt_seq != 0, 1, 0).unsqueeze(2)
# print(valid_decoder_pos.shape, valid_decoder_pos, sep='\n')

# 2,4,1 @ 2,1,4 => 2,4,4
valid_cross_pos = torch.bmm(valid_decoder_pos, valid_encoder_pos.transpose(1,2))
print("每个batch下，代表 decoder[i] 对 encoder[j] 有效性 （是否需要计算相关性）")
print(valid_cross_pos)
mask_cross_attention = (1-valid_cross_pos).to(torch.bool)
mask_cross_attention

每个batch下，代表 decoder[i] 对 encoder[j] 有效性 （是否需要计算相关性）
tensor([[[1, 1, 0, 0],
         [1, 1, 0, 0],
         [1, 1, 0, 0],
         [1, 1, 0, 0]],

        [[1, 1, 1, 1],
         [1, 1, 1, 1],
         [1, 1, 1, 1],
         [0, 0, 0, 0]]])


tensor([[[False, False,  True,  True],
         [False, False,  True,  True],
         [False, False,  True,  True],
         [False, False,  True,  True]],

        [[False, False, False, False],
         [False, False, False, False],
         [False, False, False, False],
         [ True,  True,  True,  True]]])

## Decoder self-attention-mask

decoder是一个自回归（Auto-Regressive）的模型，不是一次性输出所有结构，一次次地预测结果，可以理解为一个条件概率的生成过程。
在训练阶段，一次性送入所有数据，但是不能让模型知道所有结果，需要让模型自己预测结果。

decoder得mask是一个上三角矩阵

In [24]:
[torch.tril(torch.ones((L,L))) for L in tgt_len]

[tensor([[1., 0., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 1., 0.],
         [1., 1., 1., 1.]]),
 tensor([[1., 0., 0.],
         [1., 1., 0.],
         [1., 1., 1.]])]